# Churn modelling

In [4]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
import seaborn as sns
%matplotlib inline

In [7]:
dataset = pd.read_csv("Churn_Modelling.csv")


In [8]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
dataset.shape

(10000, 14)

In [10]:
dataset.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

# Data analysis

In [11]:
dataset[['Geography','Exited']].groupby(['Geography']).mean()

,Exited
Geography,
France,0.161548
Germany,0.324432
Spain,0.166734


churn rate of people from germany is more as compare to people from france and spain . Therefore geography have a significant affect on our output result . Therefore we will use this column in out model.

In [12]:
dataset[['Age','Exited']].groupby(['Exited']).mean()

,Age
Exited,
0,37.408389
1,44.837997


In [13]:
dataset[['Tenure','Exited']].groupby(['Exited']).mean()

,Tenure
Exited,
0,5.033279
1,4.932744


we will remove tenure because it doesn't add significance value to our model.

In [14]:
dataset[['NumOfProducts','Exited']].groupby(['NumOfProducts']).mean()

,Exited
NumOfProducts,
1,0.277144
2,0.075817
3,0.827068
4,1.000000


In [15]:
dataset[['HasCrCard','Exited']].groupby(['Exited']).mean()

,HasCrCard
Exited,
0,0.707146
1,0.699067


remove HasCrCard column

In [16]:
dataset[['IsActiveMember','Exited']].groupby(['IsActiveMember']).mean()

,Exited
IsActiveMember,
0,0.268509
1,0.142691


the column that we need to drop are 1) RowNumber   2) Customerid   3) surname   4) tenure    5) hascrcard    

In [17]:
dataset.drop(['RowNumber','CustomerId','Surname','Tenure','HasCrCard'], axis=1, inplace=True)


In [18]:
dataset.head()

,CreditScore,Geography,Gender,Age,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,0.00,1,1,101348.88,1
1,608,Spain,Female,41,83807.86,1,1,112542.58,0
2,502,France,Female,42,159660.80,3,0,113931.57,1
3,699,France,Female,39,0.00,2,0,93826.63,0
4,850,Spain,Female,43,125510.82,1,1,79084.10,0


In [19]:
dataset.columns.get_loc('Geography')

1

In [20]:
dataset

,CreditScore,Geography,Gender,Age,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,0.00,1,1,101348.88,1
1,608,Spain,Female,41,83807.86,1,1,112542.58,0
2,502,France,Female,42,159660.80,3,0,113931.57,1
3,699,France,Female,39,0.00,2,0,93826.63,0
4,850,Spain,Female,43,125510.82,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,0.00,2,0,96270.64,0
9996,516,France,Male,35,57369.61,1,1,101699.77,0
9997,709,France,Female,36,0.00,1,1,42085.58,1
9998,772,Germany,Male,42,75075.31,2,0,92888.52,1


#Data preprocessing

In [22]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler



In [23]:
features=['Geography','Gender']

In [24]:
dt=pd.get_dummies(dataset, columns=features, drop_first=True)


In [25]:
dt

,CreditScore,Age,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,0.00,1,1,101348.88,1,0,0,0
1,608,41,83807.86,1,1,112542.58,0,0,1,0
2,502,42,159660.80,3,0,113931.57,1,0,0,0
3,699,39,0.00,2,0,93826.63,0,0,0,0
4,850,43,125510.82,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
9995,771,39,0.00,2,0,96270.64,0,0,0,1
9996,516,35,57369.61,1,1,101699.77,0,0,0,1
9997,709,36,0.00,1,1,42085.58,1,0,0,0
9998,772,42,75075.31,2,0,92888.52,1,1,0,1


In [26]:
sc = MinMaxScaler()
a = sc.fit_transform(dt[['CreditScore']])
b = sc.fit_transform(dt[['Age']])
c = sc.fit_transform(dt[['Balance']])
d = sc.fit_transform(dt[['EstimatedSalary']])

In [27]:
dt['CreditScore'] = a
dt['Age'] = b
dt['Balance'] = c
dt['EstimatedSalary'] = d

In [28]:
dt

,CreditScore,Age,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,0.538,0.324324,0.000000,1,1,0.506735,1,0,0,0
1,0.516,0.310811,0.334031,1,1,0.562709,0,0,1,0
2,0.304,0.324324,0.636357,3,0,0.569654,1,0,0,0
3,0.698,0.283784,0.000000,2,0,0.469120,0,0,0,0
4,1.000,0.337838,0.500246,1,1,0.395400,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
9995,0.842,0.283784,0.000000,2,0,0.481341,0,0,0,1
9996,0.332,0.229730,0.228657,1,1,0.508490,0,0,0,1
9997,0.718,0.243243,0.000000,1,1,0.210390,1,0,0,0
9998,0.844,0.324324,0.299226,2,0,0.464429,1,1,0,1


# Resampling

In [29]:
dt.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [30]:
dt_yes=dt[dt.Exited==1]
dt_no=dt[dt.Exited==0]


In [31]:
#Before resampling
print(len(dt_yes),len(dt_no))

2037 7963


In [32]:
dt_yes_upsampled = dt_yes.sample(n=len(dt_no), replace=True, random_state=21)
print(len(dt_yes_upsampled))

7963


In [33]:
dt_upsampled=dt_no.append(dt_yes_upsampled).reset_index(drop=True)

In [34]:
dt_upsampled

,CreditScore,Age,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,0.516,0.310811,0.334031,1,1,0.562709,0,0,1,0
1,0.698,0.283784,0.000000,2,0,0.469120,0,0,0,0
2,1.000,0.337838,0.500246,1,1,0.395400,0,0,1,0
3,0.944,0.432432,0.000000,2,1,0.050261,0,0,0,1
4,0.302,0.351351,0.566170,2,1,0.374680,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
15921,0.624,0.554054,0.416776,1,0,0.040243,1,1,0,1
15922,0.520,0.432432,0.454196,2,0,0.982669,1,1,0,1
15923,0.660,0.256757,0.493452,1,0,0.408865,1,0,0,0
15924,0.340,0.378378,0.339646,1,0,0.586846,1,0,0,0


In [35]:
dt_upsampled.Exited.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

# split data into test and training set

In [36]:
from sklearn.model_selection import train_test_split
x = dt_upsampled.drop(['Exited'], axis=1) 
y = dt_upsampled['Exited']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 21)

# ann model

In [37]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [39]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 9))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(x_train, y_train, batch_size = 10, nb_epoch = 100)

C:\ProgramData\Anaconda3\envs\new\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=9, units=6, kernel_initializer="uniform")`
  """
C:\ProgramData\Anaconda3\envs\new\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
C:\ProgramData\Anaconda3\envs\new\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\envs\new\lib\site-packages\ipykernel_launcher.py:17: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
12740/12740 [==============================] - 6s 481us/step - loss: 0.6458 - accuracy: 0.6243
Epoch 2/100
12740/12740 [==============================] - 2s 180us/step - loss: 0.5913 - accuracy: 0.6879
Epoch 3/100
12740/12740 [==============================] - 3s 238us/step - loss: 0.5796 - accuracy: 0.6964
Epoch 4/100
12740/12740 [==============================] - 2s 179us/step - loss: 0.5776 - accuracy: 0.7044
Epoch 5/100
12740/12740 [==============================] - 3s 229us/step - loss: 0.5769 - accuracy: 0.7014
Epoch 6/100
12740/12740 [==============================] - 2s 190us/step - loss: 0.5769 - accuracy: 0.7021
Epoch 7/100
12740/12740 [==============================] - 3s 202us/step - loss: 0.5770 - accuracy: 0.7027
Epoch 8/100
12740/12740 [==============================] - 2s 180us/step - loss: 0.5770 - accuracy: 0.7034
Epoch 9/100
12740/12740 [==============================] - 2s 189us/step - loss: 0.5770 - accuracy: 0.7037
Epoch 10/100
12740/12740 [===========

Epoch 77/100
12740/12740 [==============================] - 2s 195us/step - loss: 0.4600 - accuracy: 0.7768
Epoch 78/100
12740/12740 [==============================] - 3s 220us/step - loss: 0.4603 - accuracy: 0.7744
Epoch 79/100
12740/12740 [==============================] - 2s 181us/step - loss: 0.4604 - accuracy: 0.7754
Epoch 80/100
12740/12740 [==============================] - 3s 207us/step - loss: 0.4603 - accuracy: 0.7768
Epoch 81/100
12740/12740 [==============================] - 2s 185us/step - loss: 0.4614 - accuracy: 0.7743
Epoch 82/100
12740/12740 [==============================] - 3s 206us/step - loss: 0.4601 - accuracy: 0.7778
Epoch 83/100
12740/12740 [==============================] - 3s 197us/step - loss: 0.4599 - accuracy: 0.7762
Epoch 84/100
12740/12740 [==============================] - 3s 227us/step - loss: 0.4604 - accuracy: 0.7745
Epoch 85/100
12740/12740 [==============================] - 2s 194us/step - loss: 0.4596 - accuracy: 0.7759
Epoch 86/100
12740/12740 [==

In [40]:
# Predicting the Test set results
y_pred = classifier.predict(x_test)
y_pred = (y_pred > 0.5)

In [41]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [42]:
cm

array([[1115,  530],
       [ 245, 1296]], dtype=int64)

In [44]:
from sklearn.metrics import accuracy_score

In [45]:
accuracy_score(y_test, y_pred)

0.7567482736974263

# random forest

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [81]:
forest_obj = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy')

In [82]:
forest_obj.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=10)

In [83]:
pred_train=forest_obj.predict(x_train)

In [84]:
accuracy_score(y_train, pred_train)

0.878806907378336

In [85]:
pred_test=forest_obj.predict(x_test)

In [86]:
accuracy_score(y_test, pred_test)

0.8449466415568111

When depth of trees is 12, the accuracy on the training set is 93.8% while that on the test set is 88.6%. The accuracy on training set is 5% higher than the accuracy on test set which indicates a slight overfitting. We can decrease the depth of a tree in the forest because as trees get deeper, they tend to be more specific which results in overfitting. However, reducing tree depth may also decrease the accuracy. So we need to be careful when optimizing the parameters. We can also increase the number of trees in the forest which will help the model to be more generalized and thus reduce overfitting. 

For reducing overfitting ,we will take depth of trees as 10.

Also for depth =10, the accuracy obtained on the training set by using Random forest classifier is 10% higher than the accuracy obtained in ANN. similarly on the test set also ,the accuracy in case of Random forest is 9% higher than that of ANN. 

This shows that Random forest performs a lot better than the ANN.